In [26]:
import gym
import pylab
import numpy as np
import random

from maxent import *

In [2]:
env = gym.make('CliffWalking-v0')

In [3]:
raw_demo = np.load(file="expert_demo/expert_trajectories.npy")

In [4]:
len(raw_demo)

5

In [35]:
n_states = env.observation_space.n # position - 20, velocity - 20
n_actions = env.nA
one_feature = env.observation_space.n # number of state per one feature
q_table = np.zeros((n_states, n_actions)) # (400, 3)
feature_matrix = np.eye((n_states)) # (400, 400)

gamma = 0.99
q_learning_rate = 0.03
theta_learning_rate = 0.05

np.random.seed(1)

In [7]:
def idx_demo(env, one_feature):
    
    raw_demo = np.load(file="expert_demo/expert_trajectories.npy")
    demonstrations = np.zeros((len(raw_demo), len(raw_demo[0])))

    for x in range(len(raw_demo)):
        for y in range(len(raw_demo[0])):
            state_idx = raw_demo[x][y][0]

            demonstrations[x][y] = state_idx
            
    return demonstrations

In [34]:
def update_q_table(state, action, reward, next_state):
    
    q_1 = q_table[state][action]
    q_2 = reward + gamma * max(q_table[next_state])
    q_table[state][action] += q_learning_rate * (q_2 - q_1)

In [36]:
demonstrations = idx_demo(env, one_feature)

expert = expert_feature_expectations(feature_matrix, demonstrations)

learner_feature_expectations = np.zeros(n_states)

theta = -(np.random.uniform(size=(n_states,)))

episodes, scores = [], []

for episode in range(30000):
    state = env.reset()
    score = 0
    print("episode" + str(episode))
    if (episode != 0 and episode == 10000) or (episode > 10000 and episode % 5000 == 0):
        learner = learner_feature_expectations / episode
        maxent_irl(expert, learner, theta, theta_learning_rate)

    while True:
        state_idx = state # idx_state(env, state)
        
        action = np.random.choice(np.where(q_table[state_idx] == q_table[state_idx].max())[0]) 
    
        #action = np.argmax(q_table[state_idx])
        next_state, reward, done, _ = env.step(action)
        
        if next_state == 47:
            done=True
        print(done, next_state, reward, action, state)
        irl_reward = get_reward(feature_matrix, theta, n_states, state_idx)
        next_state_idx = next_state # idx_state(env, next_state)
        update_q_table(state_idx, action, irl_reward, next_state_idx)

        learner_feature_expectations += feature_matrix[int(state_idx)]

        score += reward
        state = next_state

        if done:
            scores.append(score)
            episodes.append(episode)
            break

    if episode % 1000 == 0:
        score_avg = np.mean(scores)
        print('{} episode score is {:.2f}'.format(episode, score_avg))
        pylab.plot(episodes, scores, 'b')
        pylab.savefig("./learning_curves/maxent_30000.png")
        np.save("./results/maxent_q_table", arr=q_table)

episode0
False 36 -1 3 36
False 36 -100 1 36
False 24 -1 0 36
False 24 -1 3 24
False 25 -1 1 24
False 26 -1 1 25
False 25 -1 3 26
False 24 -1 3 25
False 12 -1 0 24
False 0 -1 0 12
False 12 -1 2 0
False 12 -1 3 12
False 24 -1 2 12
False 36 -1 2 24
False 36 -1 2 36
False 36 -1 3 36
False 36 -100 1 36
False 36 -1 2 36
False 24 -1 0 36
False 24 -1 3 24
False 12 -1 0 24
False 13 -1 1 12
False 25 -1 2 13
False 13 -1 0 25
False 12 -1 3 13
False 0 -1 0 12
False 0 -1 0 0
False 0 -1 3 0
False 1 -1 1 0
False 0 -1 3 1
False 1 -1 1 0
False 2 -1 1 1
False 3 -1 1 2
False 2 -1 3 3
False 2 -1 0 2
False 14 -1 2 2
False 15 -1 1 14
False 16 -1 1 15
False 28 -1 2 16
False 16 -1 0 28
False 17 -1 1 16
False 29 -1 2 17
False 36 -100 2 29
False 36 -100 1 36
False 36 -1 3 36
False 36 -1 2 36
False 24 -1 0 36
False 36 -1 2 24
False 24 -1 0 36
False 25 -1 1 24
False 36 -100 2 25
False 36 -1 2 36
False 36 -100 1 36
False 36 -1 3 36
False 24 -1 0 36
False 25 -1 1 24
False 13 -1 0 25
False 14 -1 1 13
False 2 -1 0 14

False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 14 -1 2 2
False 2 -1 0 14
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 

False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 14 -1 2 2
False 26 -1 2 14
False 27 -1 1 26
False 28 -1 1 27
False 29 -1 1 28
False 17 -1 0 29
False 29 -1 2 17
False 28 -1 3 29
False 16 -1 0 28
False 17 -1 1 16
False 18 -1 1 17
False 19 -1 1 18
False 7 -1 0 19
False 6 -1 3 7
False 18 -1 2 6
False 6 -1 0 18
False 7 -1 1 6
False 19 -1 2 7
False 31 -1 2 19
False 30 -1 3 31
False 29 -1 3 30
False 36 -100 2 29
False 24 -1 0 36
False 24 -1 3 24
False 12 -1 0 24
False 12 -1 3 12
False 0 -1 0 12
False 0 -1 0 0
False 0 -1 3 0
False 1 -1 1 0
False 0 -1 3 1
False 12 -1 2 0
False 24 -1 2 12
False 25 -1 1 24
False 13 -1 0 25
False 25 -1 2 13
False 26 -1 1 25
False 25 -1 3 26
False 36 -100 2 25
False 36 -1 2 36
False 36 -1 3 36
False 36 -100 1 36
False 24 -1 0 36
False 36 -1 2 24
False 36 -1 3 36
False 36 -100 1 36
False 36 -1 2 36
False 24 -1 0 36
False 12 -1 0 2

False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1

False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1

...

False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1

False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1

False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1

False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1 0 2
False 2 -1

KeyboardInterrupt: 